In [1]:
import exmp
from pathlib import Path
# replace os.path.join calls with Path functionality
import os.path
import qiime2
import numpy as np
import statsmodels.api as sm

import pandas as pd
import scipy as sp

from qiime2.plugins.diversity.methods import pcoa as pcoa_method, filter_distance_matrix
from qiime2.plugins.longitudinal.visualizers import anova

In [2]:
time_column = 'week'

if time_column == 'period':
    data_dir = exmp.cm_grouped_by_period_path
    sample_metadata = exmp.load_sample_metadata_grouped_by_period()
    # need to assess how these values will be used
    time_value = 1
elif time_column == 'week':
    data_dir = exmp.cm_path
    sample_metadata = exmp.load_sample_metadata()
    # need to assess how these values will be used
    time_value = '1.0'
else:
    raise ValueError("Invalid value for time_column.")

In [3]:
uu_dm = qiime2.Artifact.load(os.path.join(data_dir, "unweighted_unifrac_distance_matrix.qza"))
wu_dm = qiime2.Artifact.load(os.path.join(data_dir, "weighted_unifrac_distance_matrix.qza"))
# add bray-curtis and jaccard for consistency
faith_pd = qiime2.Artifact.load(os.path.join(data_dir, "faith_pd_vector.qza"))
shannon = qiime2.Artifact.load(os.path.join(data_dir, "shannon_vector.qza"))
evenness = qiime2.Artifact.load(os.path.join(data_dir, "evenness_vector.qza"))

base_output_dir = os.path.join(data_dir, 'ols-and-anova')


In [4]:
def ols_and_anova(dep_variable, project, time_value, base_output_dir, time_column,
                  sample_metadata, uu_dm, wu_dm, faith_pd, shannon, evenness):
    indep_variables = ['faith_pd', 'shannon', 'pielou_e', 
                       'Weighted_UniFrac_PC1', 'Weighted_UniFrac_PC2', 'Weighted_UniFrac_PC3', 
                       'Unweighted_UniFrac_PC1', 'Unweighted_UniFrac_PC2', 'Unweighted_UniFrac_PC3']
    output_dir = os.path.join(base_output_dir, '%s-%s-%s%s' % (project, dep_variable, time_column, str(time_value)))
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    where = "[project]='%s' and [%s]='%s'" % (project, time_column, str(time_value))
    
    ids_to_keep = sample_metadata.get_ids(where=where)
    sample_metadata = sample_metadata.filter_ids(ids_to_keep=ids_to_keep)

    # make column names compatible for R-like forumulas used in anova
    _df = sample_metadata.to_dataframe()
    _df.index.name = 'sample-id'
    _df = _df.rename(columns={'VO2max-change': 'VO2max_change',
                              'RER-change': 'RER_change',
                              'row-change': 'row_change',
                              'bench-press-change': 'bench_press_change',
                              '3RM-squat-change': 'three_rep_max_squat_change'})

    # drop columns that don't have necessary data
    if project == 'exmp1':
        _df = _df[['VO2max_change', 'RER_change']].dropna().astype(np.float)
    elif project == 'exmp2':
        _df = _df[['row_change', 'bench_press_change', 'three_rep_max_squat_change']].dropna().astype(np.float)
    else:
        raise ValueError("Project must be exmp1 or exmp2, but %s was provided." % project)
    sample_metadata = qiime2.Metadata(_df)

    uu_dm = filter_distance_matrix(uu_dm, metadata=sample_metadata).filtered_distance_matrix
    wu_dm = filter_distance_matrix(wu_dm, metadata=sample_metadata).filtered_distance_matrix

    wu_pcoa = pcoa_method(wu_dm).pcoa
    wu_pcoa = wu_pcoa.view(qiime2.Metadata).to_dataframe()[['Axis 1', 'Axis 2', 'Axis 3']]
    wu_pcoa = wu_pcoa.rename(columns={'Axis 1': 'Weighted_UniFrac_PC1', 
                                      'Axis 2': 'Weighted_UniFrac_PC2', 
                                      'Axis 3': 'Weighted_UniFrac_PC3'})
    sample_metadata = sample_metadata.merge(qiime2.Metadata(wu_pcoa))
    
    uu_pcoa = pcoa_method(uu_dm).pcoa
    uu_pcoa = uu_pcoa.view(qiime2.Metadata).to_dataframe()[['Axis 1', 'Axis 2', 'Axis 3']]
    uu_pcoa = uu_pcoa.rename(columns={'Axis 1': 'Unweighted_UniFrac_PC1', 
                                      'Axis 2': 'Unweighted_UniFrac_PC2', 
                                      'Axis 3': 'Unweighted_UniFrac_PC3'})
    sample_metadata = sample_metadata.merge(qiime2.Metadata(uu_pcoa))

    sample_metadata = sample_metadata.merge(faith_pd.view(qiime2.Metadata))
    sample_metadata = sample_metadata.merge(shannon.view(qiime2.Metadata))
    sample_metadata = sample_metadata.merge(evenness.view(qiime2.Metadata))


    df = sample_metadata.to_dataframe()
    df = sm.add_constant(df)
    
    dep_variable_histogram = df[dep_variable].hist().figure
    dep_variable_histogram.savefig(os.path.join(output_dir, 'histogram.pdf'))

    mod = sm.OLS(df[dep_variable], df[['const'] + indep_variables])
    res = mod.fit()
    ols_result_summary = res.summary()
    with open(os.path.join(output_dir, 'ols.txt'), 'w') as fh:
        fh.write(ols_result_summary.as_text())
    
    formula = "%s ~ %s" % (dep_variable, ' + '.join(indep_variables))
    anova_visualization = anova(metadata=qiime2.Metadata(df), formula=formula).visualization
    anova_visualization.save(os.path.join(output_dir, 'anova.qzv'))
    
    return dep_variable_histogram, ols_result_summary, anova_visualization, sample_metadata
    
    

In [5]:
r1 = ols_and_anova('VO2max_change', 'exmp1', time_value, base_output_dir, time_column,
                   sample_metadata, uu_dm, wu_dm, faith_pd, shannon, evenness)
r2 = ols_and_anova('RER_change', 'exmp1', time_value, base_output_dir, time_column,
                   sample_metadata, uu_dm, wu_dm, faith_pd, shannon, evenness)
r3 = ols_and_anova('row_change', 'exmp2', time_value, base_output_dir, time_column,
                   sample_metadata, uu_dm, wu_dm, faith_pd, shannon, evenness)
r4 = ols_and_anova('bench_press_change', 'exmp2', time_value, base_output_dir, time_column,
                   sample_metadata, uu_dm, wu_dm, faith_pd, shannon, evenness)
r5 = ols_and_anova('three_rep_max_squat_change', 'exmp2', time_value, base_output_dir, time_column,
                   sample_metadata, uu_dm, wu_dm, faith_pd, shannon, evenness)

/Users/gregcaporaso/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.04916905343409495 and the largest is 2.0003670932524003.
  RuntimeWarning
/Users/gregcaporaso/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/gregcaporaso/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result c

In [6]:
print(r1[1])

                            OLS Regression Results                            
Dep. Variable:          VO2max_change   R-squared:                       0.350
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     1.136
Date:                Wed, 12 Feb 2020   Prob (F-statistic):              0.386
Time:                        17:19:33   Log-Likelihood:                -64.961
No. Observations:                  29   AIC:                             149.9
Df Residuals:                      19   BIC:                             163.6
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -13

In [7]:
print(r1[2])

<visualization: Visualization uuid: 950317b7-c1aa-4b8b-ab72-8391f8d7c78a>


In [8]:
print(r4[1])

                            OLS Regression Results                            
Dep. Variable:     bench_press_change   R-squared:                       0.578
Model:                            OLS   Adj. R-squared:                 -0.182
Method:                 Least Squares   F-statistic:                    0.7600
Date:                Wed, 12 Feb 2020   Prob (F-statistic):              0.661
Time:                        17:20:08   Log-Likelihood:                -37.211
No. Observations:                  15   AIC:                             94.42
Df Residuals:                       5   BIC:                             101.5
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -16